In [66]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from geopy.distance import geodesic

from sklearn.model_selection import train_test_split

In [73]:
RANDOM_STATE = 42

In [ ]:
data = pd.read_excel('../../data/fraud.xlsx')
# Data Cleaning Steps
data['trans_date_trans_time'] = pd.to_datetime(data['trans_date_trans_time'], errors='coerce')
data['dob'] = pd.to_datetime(data['dob'], errors='coerce')
data['is_fraud'] = data['is_fraud'].astype(str).str.replace('1"2020-12-24 16:56:24"', '1')
data['is_fraud'] = data['is_fraud'].str.replace('0"2019-01-01 00:00:44"', '0')
data['is_fraud'] = data['is_fraud'].astype(int)
data['merchant'] = data['merchant'].str.replace(r'^"|"$', '', regex=True)
data['job'] = data['job'].str.replace(r'^"|"$', '', regex=True)
data = data.drop_duplicates(subset='trans_num', keep='first')
data = data.sort_values(by='trans_date_trans_time')

data

,trans_date_trans_time,merchant,category,amt,city,state,lat,long,city_pop,job,dob,trans_num,merch_lat,merch_long,is_fraud
7781,2019-01-01 00:00:00,Lind-Buckridge,entertainment,220.11,Malad City,ID,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,43.150704,-112.154481,0
7782,2019-01-01 00:07:00,Kiehn Inc,grocery_pos,96.29,Grenada,CA,41.6125,-122.5258,589,Systems analyst,1945-12-21,413636e759663f264aae1819a4d4f231,41.657520,-122.230347,0
7783,2019-01-01 00:09:00,Beier-Hyatt,shopping_pos,7.77,High Rolls Mountain Park,NM,32.9396,-105.8189,899,Naval architect,1967-08-30,8a6293af5ed278dea14448ded2685fea,32.863258,-106.520205,0
7784,2019-01-01 00:21:00,Bruen-Yost,misc_pos,6.85,Freedom,WY,43.0172,-111.0292,471,"Education officer, museum",1967-02-08,f3c43d336e92a44fc2fb67058d5949e3,43.753735,-111.454923,0
7785,2019-01-01 00:22:00,Kunze Inc,grocery_pos,90.22,Honokaa,HI,20.0827,-155.4880,4878,Physiotherapist,1966-03-12,95826e3caa9e0b905294c6dae985aec1,19.560013,-156.045889,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7775,2020-12-31 23:57:00,"Larkin, Stracke and Greenfelde",entertainment,46.71,Blairsden-Graeagle,CA,39.8127,-120.6405,1725,Chartered legal executive (England and Wales),1967-05-27,a7105564935ea3977dc61ff9ced3bf5e,38.963543,-120.457121,0
7777,2020-12-31 23:58:00,"Pouros, Walker and Spence",kids_pets,13.02,Greenview,CA,41.5403,-122.9366,308,Call centre manager,1958-09-20,bd7071fd5c9510a5594ee196368ac80e,41.973127,-123.553032,0
7780,2020-12-31 23:59:00,Breitenberg LLC,travel,7.99,Mesa,ID,44.6255,-116.4493,129,Cartographer,1965-12-15,14392d723bb7737606b2700ac791b7aa,44.470525,-117.080888,0
7778,2020-12-31 23:59:00,Reilly and Sons,health_fitness,43.77,Luray,MO,40.4931,-91.8912,519,Town planner,1966-02-13,9b1f753c79894c9f4b71f04581835ada,39.946837,-91.333331,0


In [84]:
X = data.drop(columns='is_fraud')
y = data['is_fraud']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)
print(f'Train Distribution:')
print(f'{y_train.value_counts(normalize=True)} \n')
print(f'Validation Distribution:')
print(f'{y_val.value_counts(normalize=True)} \n')

Train Distribution:
is_fraud
0    0.876065
1    0.123935
Name: proportion, dtype: float64 

Validation Distribution:
is_fraud
0    0.87626
1    0.12374
Name: proportion, dtype: float64 

